<a href="https://colab.research.google.com/github/joiedevivre02/joiedevivre02/blob/main/prepare_instantly_campaigns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Project info
#Combine leads info into a single tracker

In [ ]:
#@title Initialize
import pandas as pd
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


In [ ]:
#@title Enter url of source file
#this contains all the links to the files and sheet names
#replace link
# url = 'https://docs.google.com/spreadsheets/d/1tZGu3GBvl2qPGh0yluc_mtWZl7CKhi4b7jfVrTUFwzk/edit#gid=62501885'
# url = 'https://docs.google.com/spreadsheets/d/1py1TBG3M0gSGnS6efIHl2fIKUqq8KhSHsUD1kKsD6yU/edit#gid=507012045'
url = 'https://docs.google.com/spreadsheets/d/1Jzc8_sPWrzPcEK21o5lLQ1m_7vq1g0UbBajKGxCZEb0/edit#gid=0'
wb = gc.open_by_url(url)

In [ ]:
#@title Open the Google Sheet file you want to write your new data in and then insert the data with the set function
from gspread_dataframe import get_as_dataframe, set_with_dataframe

def save_to_sheets(name_of_sheet, dataframe, row_number, col_number, url):
    wb3 = gc.open_by_url(url)
    sheet_output = wb3.worksheet(name_of_sheet)
    set_with_dataframe(sheet_output, dataframe, row=row_number, col=col_number)


In [ ]:
#@title Read source and convert to dataframe

def read_df(worksheet, sheet_name):
    df = worksheet.worksheet(sheet_name)
    df = df.get_all_values()
    df = pd.DataFrame(df)
    df.columns = df.iloc[0].str.upper()
    df = df.iloc[1:]
    return df


In [ ]:
#@title Part II- Workbooks with conditional notes

# all_sheet = read_df(url=workbook_name_links['Completed - 2024.03 - Landscapers
# - Multivariable'].iloc[0], sheet_name='Chase')
#Concatenate all sheets and add source_name column to indicate source
# Get a list of all spreadsheets
def combine_sheets(url):
      # Open the spreadsheet by its ID
      wb2 = gc.open_by_url(url)

      # Get all worksheets in the spreadsheet
      worksheets = wb2.worksheets()

      # Initialize a list to hold all DataFrames from all worksheets
      dfs = []

      # Iterate over each worksheet, retrieve all values, and append them to dfs
      for worksheet in worksheets:
          values = worksheet.get_all_values()
          df = pd.DataFrame(values[1:], columns=values[0]) # Skip the header row
          df['Sheet'] = worksheet.title # Add a column for the sheet name
          dfs.append(df)
      # Concatenate all DataFrames into a single DataFrame
      final_df = pd.concat(dfs, ignore_index=True)

      return final_df


In [ ]:
import pandas as pd

# Assuming df is your DataFrame
def check_first_name_condition(row):
    if pd.isnull(row['first_name']) or row['first_name'].lower() == 'there':
        return True
    else:
        return False

def first_name_flex(df):

     # Add a new column "First_name_flex" based on the conditions
     df['first_name_flex'] = np.where(
        (df['first_name'].isnull()) | (df['first_name'] == 'there'),
        df['cleaned_company_name'],
        df['first_name'])
     return df



In [ ]:
#@title Step 1

workbook="https://docs.google.com/spreadsheets/d/1Jzc8_sPWrzPcEK21o5lLQ1m_7vq1g0UbBajKGxCZEb0/edit#gid=0"
combined = combine_sheets(workbook)
# Add a new column 'Condition' to the DataFrame
combined['first_name'] = combined['first_name'].astype('string')

combined['first_name_null'] = combined.apply(check_first_name_condition, axis=1)

#check previous copy of campaign variables
# tavus_thumbnail_url
# tavus_video

#replace fields
combined.rename(columns={"url_with_query_string": "tavus_video", "gif_thumbnail_url":"tavus_thumbnail_url"}, inplace=True)
combined['tavus_video'] = combined['tavus_video'].astype('string')

#tavus video is the column referring to different columns


In [ ]:
#check how many are null video
empty_string_counts = combined[combined['tavus_video'] == ""].groupby('Sheet').size().reset_index(name='counts')
print(empty_string_counts)


                  Sheet  counts
0   David - Retargeting      14
1  Eva - Website Script       3


In [ ]:
combined['Sheet'].unique()

array(['David - Website Script', 'David - Retargeting',
       'Eva - Website Script'], dtype=object)

In [ ]:
#check if first_name is not null
combined['first_name_null'].unique()

array([ True])

In [ ]:
select_sheets = ['David - Website Script', 'David - Retargeting']
combined=combined[combined['Sheet'].isin(select_sheets)]

In [ ]:
#@title Remove null tavus videos and update tracker

combined_all = combined.copy()

#remove with null tavus video and add first_name_flex
with_video=combined_all[combined_all['tavus_video']!=""]

import numpy as np
with_video_flex=first_name_flex(with_video)


columns_to_keep_tracker = [
'id',
'batch_name',
'cleaned_company_name',
'industry',
'phone_number',
'email',
'first_name_flex',
'first_name'
]

import numpy as np
with_video_select = with_video_flex[columns_to_keep_tracker]



#SAVE TO RLDB ASSIGNED TRACKER
output_workbook = "https://docs.google.com/spreadsheets/d/18nCENIA-4bIHW0b4B2R-yJEKz8JbBb41uaCnb0mCsjc/edit#gid=1407520858"

save_to_sheets("WIP", dataframe=with_video_select, row_number=1, col_number=1, url=output_workbook)




<ipython-input-6-fd3d55082687>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_name_flex'] = np.where(


In [ ]:


columns_to_keep = ['city',
 'email',
 'state',
 'industry',
 'prompt_1',
 'prompt_2',
 #'phone',
 'first_name',
 'rep_full_name',
 'rep_first_name',
 'cleaned_company_name',
 'tavus_thumbnail_url',
 'tavus_video']


result_df = with_video[columns_to_keep]
output_workbook = "https://docs.google.com/spreadsheets/d/1Jzc8_sPWrzPcEK21o5lLQ1m_7vq1g0UbBajKGxCZEb0/edit#gid=0"

save_to_sheets("Sheet1", dataframe=result_df, row_number=1, col_number=1, url=output_workbook)



In [ ]:
#@title Split file into with first_name, and without


#without first name
combined_subset = with_video[with_video['first_name_null']==True]

#with first name
# combined_subset = with_video[with_video['first_name_null']==False]

richard = combined_subset[combined_subset['Sheet']=="richard"]
dvaire = combined_subset[combined_subset['Sheet']=="dvaire"]
patrick  = combined_subset[combined_subset['Sheet']=="patrick"]
jd = combined_subset[combined_subset['Sheet']=="jd"]
eva = combined_subset[combined_subset['Sheet']=="eva"]
isabel = combined_subset[combined_subset['Sheet']=="isabel"]
david = combined_subset[combined_subset['Sheet']=="david"]
pete = combined_subset[combined_subset['Sheet']=="pete"]
chase = combined_subset[combined_subset['Sheet']=="chase"]
dan = combined_subset[combined_subset['Sheet']=="dan"]

In [ ]:
#recheck empty url

#check how many are null video
empty_string_counts = combined_subset[combined_subset['tavus_video'] == ""].groupby('Sheet').size().reset_index(name='counts')
print(empty_string_counts)


Empty DataFrame
Columns: [Sheet, counts]
Index: []


In [ ]:
#combined_filtered = combined[columns_to_keep]
richard = richard[columns_to_keep]
dvaire = dvaire[columns_to_keep]
patrick  = patrick[columns_to_keep]
jd = jd[columns_to_keep]
eva = eva[columns_to_keep]
isabel = isabel[columns_to_keep]
david = david[columns_to_keep]
pete = pete[columns_to_keep]
chase = chase[columns_to_keep]
dan = dan[columns_to_keep]



In [ ]:
#@title Export Concatenated tables

#without_first_name
output_workbook = "https://docs.google.com/spreadsheets/d/1-2y080pxvdx5pX2NY65tuINXXWTEP_QY5x7IwjAMDLo/edit#gid=0"

#with_first_name
# output_workbook = "https://docs.google.com/spreadsheets/d/1kpFmQt54ObcYWfjTo0hFOb-MINEKJk8unHg9phHjjAQ/edit#gid=187673915"



save_to_sheets("richard", dataframe=richard, row_number=1, col_number=1, url=output_workbook)
save_to_sheets("dvaire", dataframe=dvaire, row_number=1, col_number=1, url=output_workbook)
save_to_sheets("patrick", dataframe=patrick, row_number=1, col_number=1, url=output_workbook)
save_to_sheets("jd", dataframe=jd, row_number=1, col_number=1, url=output_workbook)
save_to_sheets("eva", dataframe=eva, row_number=1, col_number=1, url=output_workbook)
save_to_sheets("isabel", dataframe=isabel, row_number=1, col_number=1, url=output_workbook)
save_to_sheets("david", dataframe=david, row_number=1, col_number=1, url=output_workbook)
save_to_sheets("pete", dataframe=pete, row_number=1, col_number=1, url=output_workbook)
save_to_sheets("chase", dataframe=chase, row_number=1, col_number=1, url=output_workbook)
save_to_sheets("dan", dataframe=dan, row_number=1, col_number=1, url=output_workbook)